In [2]:
import pandas as pd
import random
import numpy as np
import seaborn as sns
import math
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression,Lasso
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

In [3]:
X=pd.read_csv('x_train.csv',delimiter=";",header=None,names=['f'+str(i) for i in range (223)])
X_test=pd.read_csv('x_test.csv',delimiter=";",header=None,names=['f'+str(i) for i in range (223)])
y=pd.read_csv('y_train.csv',names=['class'])
y=y['class']

In [3]:
X_ftr=pd.read_csv('ftr_train.csv')
X_ftr.drop('Unnamed: 0',inplace=True,axis=1)

Добавим булевские фичи

In [6]:
cols=['f11', 'f18', 'f27', 'f63', 'f72', 'f98', 'f108', 'f112', 'f114', 'f138', 'f168', 'f170', 'f171', 'f179', 'f188', 'f205', 'f213']
for c in cols:
    X['b'+c]=(X[c]>0)*1

In [26]:
# Фуникция считает качество модели на кроссвалилации
# clf - модель, X,y - данные, ftr_set - пподмножество названий колонок, на которых считаем модель
# cv_val - количество холдов для кросс валидации
def CV_score(clf,X,y,ftr_set,cv_val):
    # Если подмоножество признаков пустое, то возвращаем 0 
    if len(ftr_set)==0:
        return 0
    # Иначе делаем кроссвалидацию
    scores = cross_val_score(clf, X[ftr_set],y,cv=cv_val)
    return scores.mean()
    #return random.random()

# Функция определяет  фичу, добавление которой наилучшим образом увеличивает качество 
# Фуникция берет случайную фичу из диапазона, добавляет её к стартовому набору фич и строит качество
# N -  максимальное количество итераций
# start_ftr - стартовый набор признаков
# start_cv - стартовое значение качества
def get_best_ftr(clf,X,y,start_ftr,start_cv,cv_val,N_params):
    print ("===START ftr lookup")
    print ("Start_ftr_set=",start_ftr)
    print ("Start_cv=     ",start_cv)
    i=0
    max_iter=N_params[0]
    start_ftr_N=N_params[1]
    end_ftr_N=N_params[2]
    used_ftr=start_ftr.copy()
    best_cv=start_cv
    best_ftr=''
    if max_iter==99999:
        i=start_ftr_N
    feature_names=list(X.columns)
    while i<max_iter:
        if max_iter<99999:
            # выбираем случайную фичу
            ftr_index=random.randint(start_ftr_N,end_ftr_N)
        else:
            ftr_index=i
            i=i+1
        ftr_name=feature_names[ftr_index]
        # Проверяем, что её не использвоали
        if ftr_name in used_ftr:
            continue
        # Считаем качество на наборе фич с учетом выбранной
        ftr_set=start_ftr.copy()
        ftr_set.append(ftr_name)
        cv=CV_score(clf,X,y,ftr_set,cv_val)
        if cv>best_cv:
            best_cv=cv
            best_ftr=ftr_name
            print ("  ! ",end='')
        else:
            print ("    ",end='')
        print ("{:5} {:10}".format(i,ftr_name), " ",cv)
        used_ftr.append(ftr_name)
        if (max_iter<99999):
            i=i+1
        if (max_iter==99999)&(i>end_ftr_N):
            i=99999+1
    print ("===END ftr lookup ")
    return best_ftr,best_cv

# Фуникция производит отбор фич методом Sequential Feature Selection
def sfs(clf,X,y,start_ftr,cv_val,N_params):
    cur_ftr=start_ftr.copy()
    best_cv=CV_score(clf,X,y,start_ftr,cv_val)
    doAdd=True
    while doAdd:
        new_ftr,best_cv=get_best_ftr(clf,X,y,cur_ftr,best_cv,cv_val,N_params)
        if (new_ftr!=''):
            print (new_ftr+" added! New cv=",best_cv)
            cur_ftr.append(new_ftr)
        else: doAdd=False
    print ("Final_ftr_set=",cur_ftr, " final_cv=",best_cv)
    return cur_ftr,best_cv

In [48]:
clf = RandomForestClassifier(max_features='auto',n_estimators=300,max_depth=100,random_state=42)
start_ftr= ['f138', 'f96', 'f76', 'f11', 'f185', 'f115', 'f131','f83', 'bf11'] 
max_iter=99999
start_ftr_N=240
end_ftr_N=249
N_params=(max_iter, start_ftr_N,end_ftr_N)
best_ftr,best_cv=sfs(clf,X,y,start_ftr,5,N_params)

===START ftr lookup
Start_ftr_set= ['f138', 'f96', 'f76', 'f11', 'f185', 'f115', 'f131', 'f83', 'bf11']
Start_cv=      0.636874370739
  !   241 pf18         0.998567742815
      242 pf27         0.998280381788
  !   243 pf72         0.999142038477
      244 pf98         0.997993437742
      245 pf112        0.998282023824
      246 pf114        0.998282434918
      247 pf168        0.99770649722
      248 pf170        0.998854682155
      249 pf171        0.998278333362
      250 pf205        0.9982824361
===END ftr lookup 
pf72 added! New cv= 0.999142038477
===START ftr lookup
Start_ftr_set= ['f138', 'f96', 'f76', 'f11', 'f185', 'f115', 'f131', 'f83', 'bf11', 'pf72']
Start_cv=      0.999142038477
      241 pf18         0.99856856148
      242 pf27         0.998853040119
      244 pf98         0.997707723456
      245 pf112        0.999140396441
  !   246 pf114        0.999427752763
      247 pf168        0.999141219812
      248 pf170        0.998568149204
      249 pf171        0.998

KeyboardInterrupt: 

In [16]:
X.columns[239]

'bf213'

In [31]:
# Предсказываем фичи

In [37]:
ftr_to_pred=['f18','f27','f72','f98','f112','f114','f168','f170','f171','f205']
for ftr in ftr_to_pred:
    clf=RandomForestClassifier(n_estimators=60)
    clf.fit(X[[ftr]],y)
    score=clf.score(X[[ftr]],y)
    print (ftr,score)
    X['p'+ftr]=clf.predict(X[[ftr]])

f18 0.99856692462
f27 0.99856692462
f72 0.999426769848
f98 0.997993694468
f112 0.99856692462
f114 0.99856692462
f168 0.997707079392
f170 0.999140154772
f171 0.998280309544
f205 0.99856692462


In [46]:
X.columns[249]

'pf205'

In [50]:
print (np.mean(cross_val_score(clf, X[['f27']], y, cv=5)))

0.37462519453
